In [ ]:
"""
CSIRO Biomass Competition - Insight Engine Runner
==================================================
Adapted to handle the long-format train.csv structure and extract
biomass-specific patterns.
"""

import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import the InsightEngine (assumes previous code is saved or in same file)
# For standalone execution, paste the InsightEngine class code above this

def load_and_transform_biomass_data(csv_path):
    """
    Load CSIRO train.csv and transform from long to wide format.
    
    Args:
        csv_path: Path to train.csv
        
    Returns:
        df_wide: DataFrame with one row per image
        df_long: Original long-format DataFrame
    """
    print("📁 Loading CSIRO train.csv...")
    df_long = pd.read_csv(csv_path)
    
    print(f"   Loaded: {len(df_long)} rows (long format)")
    print(f"   Unique images: {df_long['image_path'].nunique()}")
    
    # Extract image ID from image_path for grouping
    df_long['image_id'] = df_long['image_path'].str.extract(r'ID(\d+)')[0]
    
    # Pivot: One row per image, columns for each target
    print("\n🔄 Transforming to wide format...")
    df_wide = df_long.pivot_table(
        index=['image_id', 'image_path', 'Sampling_Date', 'State', 'Species', 
               'Pre_GSHH_NDVI', 'Height_Ave_cm'],
        columns='target_name',
        values='target'
    ).reset_index()
    
    # Flatten column names
    df_wide.columns.name = None
    
    print(f"   Wide format: {len(df_wide)} rows × {len(df_wide.columns)} columns")
    print(f"   Target columns: {[col for col in df_wide.columns if col.startswith('Dry_') or col == 'GDM_g']}")
    
    return df_wide, df_long


def engineer_biomass_features(df):
    """
    Create domain-specific features for biomass prediction.
    """
    print("\n🧬 Engineering biomass-specific features...")
    
    df = df.copy()
    
    # 1. Date features
    df['Sampling_Date'] = pd.to_datetime(df['Sampling_Date'])
    df['Month'] = df['Sampling_Date'].dt.month
    df['Day_of_Year'] = df['Sampling_Date'].dt.dayofyear
    df['Season'] = df['Month'].map({
        12: 'Summer', 1: 'Summer', 2: 'Summer',
        3: 'Autumn', 4: 'Autumn', 5: 'Autumn',
        6: 'Winter', 7: 'Winter', 8: 'Winter',
        9: 'Spring', 10: 'Spring', 11: 'Spring'
    })
    
    # Cyclical encoding for time
    df['Month_Sin'] = np.sin(2 * np.pi * df['Month'] / 12)
    df['Month_Cos'] = np.cos(2 * np.pi * df['Month'] / 12)
    df['DOY_Sin'] = np.sin(2 * np.pi * df['Day_of_Year'] / 365)
    df['DOY_Cos'] = np.cos(2 * np.pi * df['Day_of_Year'] / 365)
    
    # 2. Biomass ratios (key predictive signals from research)
    df['Dead_to_Total_Ratio'] = df['Dry_Dead_g'] / (df['Dry_Total_g'] + 1e-6)
    df['Clover_to_Total_Ratio'] = df['Dry_Clover_g'] / (df['Dry_Total_g'] + 1e-6)
    df['Green_to_Total_Ratio'] = df['Dry_Green_g'] / (df['Dry_Total_g'] + 1e-6)
    df['GDM_to_Total_Ratio'] = df['GDM_g'] / (df['Dry_Total_g'] + 1e-6)
    
    # 3. NDVI-based features
    df['NDVI_x_Height'] = df['Pre_GSHH_NDVI'] * df['Height_Ave_cm']
    df['Biomass_per_NDVI'] = df['Dry_Total_g'] / (df['Pre_GSHH_NDVI'] + 0.1)
    df['Biomass_per_Height'] = df['Dry_Total_g'] / (df['Height_Ave_cm'] + 1)
    
    # 4. Binary flags
    df['Has_Clover'] = (df['Dry_Clover_g'] > 0).astype(int)
    df['Has_Dead'] = (df['Dry_Dead_g'] > 0).astype(int)
    df['Is_WA'] = (df['State'] == 'WA').astype(int)
    
    # 5. Species encoding
    df['Species_Count'] = df['Species'].str.count('_') + 1  # Number of species
    df['Has_Ryegrass'] = df['Species'].str.contains('Ryegrass', case=False).astype(int)
    df['Has_Clover_Species'] = df['Species'].str.contains('Clover|clover', case=False).astype(int)
    df['Has_Lucerne'] = df['Species'].str.contains('Lucerne', case=False).astype(int)
    
    # 6. State encoding (ordinal by typical biomass productivity)
    state_map = {'Tas': 0, 'Vic': 1, 'NSW': 2, 'SA': 3, 'WA': 4, 'QLD': 5}
    df['State_Code'] = df['State'].map(state_map).fillna(0)
    
    print(f"   Created {len([c for c in df.columns if c not in ['image_id', 'image_path', 'Sampling_Date', 'State', 'Species', 'Season']])} numeric features")
    
    return df


def run_biomass_analysis(csv_path='/kaggle/input/csiro-biomass/train.csv', 
                         output_dir='biomass_insights'):
    """
    Complete analysis pipeline for CSIRO biomass data.
    """
    print("="*70)
    print("🌾 CSIRO BIOMASS COMPETITION - AUTOMATED INSIGHT DISCOVERY")
    print("="*70)
    
    # Step 1: Load and transform data
    df_wide, df_long = load_and_transform_biomass_data(csv_path)
    
    # Step 2: Engineer features
    df_analysis = engineer_biomass_features(df_wide)
    
    # Step 3: Define columns for analysis
    # Exclude identifiers and categorical text columns
    exclude_cols = [
        'image_id', 'image_path', 'Sampling_Date', 'State', 'Species', 'Season'
    ]
    
    # Target columns
    target_cols = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']
    
    # Step 4: Run analysis for each target
    all_insights = {}
    
    for target in target_cols:
        print("\n" + "="*70)
        print(f"🎯 ANALYZING TARGET: {target}")
        print("="*70)
        
        # Create engine instance
        engine = InsightEngine(output_dir=f'{output_dir}/{target}')
        
        # Run analysis
        insights = engine.analyze(
            df_analysis,
            target_col=target,
            id_cols=exclude_cols
        )
        
        # Generate report
        engine.generate_report(f'{target}_report.txt')
        
        all_insights[target] = insights
    
    # Step 5: Cross-target analysis
    print("\n" + "="*70)
    print("🔬 CROSS-TARGET PATTERN ANALYSIS")
    print("="*70)
    
    analyze_target_relationships(df_analysis, target_cols, output_dir)
    
    # Step 6: Investigate WA Dead Matter Anomaly
    investigate_wa_anomaly(df_analysis, output_dir)
    
    # Step 7: Zero-inflation analysis
    analyze_zero_inflation(df_analysis, target_cols, output_dir)
    
    print("\n" + "="*70)
    print("✅ COMPLETE ANALYSIS FINISHED")
    print("="*70)
    print(f"\n📂 All outputs saved to: {output_dir}/")
    print(f"\n💡 KEY FINDINGS TO CHECK:")
    print("   1. Feature importance rankings for each target")
    print("   2. WA Dead Matter investigation results")
    print("   3. Zero-inflation patterns")
    print("   4. Cluster profiles (species/state groupings)")
    print("   5. Interaction features (NDVI × Height, ratios)")
    
    return all_insights, df_analysis


def analyze_target_relationships(df, target_cols, output_dir):
    """
    Analyze how the five targets relate to each other.
    """
    print("\n📊 Target Correlation Matrix:")
    target_corr = df[target_cols].corr()
    print(target_corr.round(3))
    
    # Check hierarchical consistency
    print("\n🔍 Checking Hierarchical Consistency:")
    df_check = df.copy()
    df_check['GDM_calculated'] = df_check['Dry_Green_g'] + df_check['Dry_Clover_g']
    df_check['Total_calculated'] = df_check['GDM_g'] + df_check['Dry_Dead_g']
    
    gdm_error = (df_check['GDM_g'] - df_check['GDM_calculated']).abs().mean()
    total_error = (df_check['Dry_Total_g'] - df_check['Total_calculated']).abs().mean()
    
    print(f"   GDM = Green + Clover: MAE = {gdm_error:.4f}g")
    print(f"   Total = GDM + Dead: MAE = {total_error:.4f}g")
    
    if gdm_error > 1.0 or total_error > 1.0:
        print("   ⚠️  Warning: Hierarchical constraints not perfectly satisfied!")
    else:
        print("   ✅ Hierarchical constraints validated")


def investigate_wa_anomaly(df, output_dir):
    """
    Investigate the Western Australia dead biomass anomaly.
    """
    print("\n🔬 INVESTIGATING WA DEAD MATTER ANOMALY:")
    print("-" * 60)
    
    # Group by state
    state_stats = df.groupby('State').agg({
        'Dry_Dead_g': ['mean', 'std', 'min', 'max', lambda x: (x == 0).sum()],
        'Dry_Total_g': 'mean',
        'Pre_GSHH_NDVI': 'mean',
        'Height_Ave_cm': 'mean'
    }).round(2)
    
    print("\nDead Biomass by State:")
    print(state_stats)
    
    # WA specific analysis
    wa_data = df[df['State'] == 'WA']
    wa_dead_zero_pct = (wa_data['Dry_Dead_g'] == 0).sum() / len(wa_data) * 100
    
    print(f"\n📍 Western Australia:")
    print(f"   Samples: {len(wa_data)}")
    print(f"   Dead = 0: {wa_dead_zero_pct:.1f}% of samples")
    print(f"   Mean NDVI: {wa_data['Pre_GSHH_NDVI'].mean():.3f}")
    print(f"   Mean Height: {wa_data['Height_Ave_cm'].mean():.2f} cm")
    
    if wa_dead_zero_pct > 80:
        print("\n   ⚠️  CONFIRMED: WA shows systematic zero dead matter")
        print("   💡 RECOMMENDATION: Use state-conditional model or post-processing")
        print("      → IF State == WA: Set Dry_Dead_g = 0")


def analyze_zero_inflation(df, target_cols, output_dir):
    """
    Analyze zero-inflation patterns in targets.
    """
    print("\n📉 ZERO-INFLATION ANALYSIS:")
    print("-" * 60)
    
    for target in target_cols:
        zero_pct = (df[target] == 0).sum() / len(df) * 100
        zero_count = (df[target] == 0).sum()
        
        # Statistical properties
        mean_val = df[target].mean()
        std_val = df[target].std()
        cv = std_val / (mean_val + 1e-6)  # Coefficient of variation
        
        # Distribution shape
        skewness = df[target].skew()
        
        print(f"\n{target}:")
        print(f"   Zero values: {zero_count} ({zero_pct:.1f}%)")
        print(f"   Mean: {mean_val:.2f}g, StdDev: {std_val:.2f}g")
        print(f"   CV: {cv:.2f}, Skewness: {skewness:.2f}")
        
        if zero_pct > 20:
            print(f"   ⚠️  HIGH zero-inflation detected")
            print(f"   💡 Recommendation: Use Tweedie Loss (p=1.5) or ZIG model")
        elif zero_pct > 5:
            print(f"   ⚡ MODERATE zero-inflation")
            print(f"   💡 Recommendation: Consider Tweedie Loss")
        else:
            print(f"   ✅ Low zero-inflation, standard losses OK")


# Main execution
if __name__ == "__main__":
    # Run the complete analysis
    insights, df_processed = run_biomass_analysis(
        csv_path='/kaggle/input/csiro-biomass/train.csv',
        output_dir='biomass_insights'
    )
    
    print("\n🎉 Analysis complete! Review the outputs to guide your modeling strategy.")
    print("\n📋 NEXT STEPS:")
    print("   1. Review feature importance rankings → Focus model on top features")
    print("   2. Check cluster profiles → Consider separate models per cluster")
    print("   3. Use interaction features → Add to your DINOv2 metadata fusion")
    print("   4. Handle WA anomaly → Apply state-conditional post-processing")
    print("   5. Select appropriate loss → Tweedie for Clover/Dead, MSE for Total")